In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# #이후 런타임 다시 시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,948 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [ ]:
import warnings   
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams["font.size"] = 12
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import numpy as np
import pandas as pd
import re


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [ ]:
cd /content/drive/MyDrive/UBION_final_project/data

/content/drive/.shortcut-targets-by-id/1cbruNkx_Sh7hGut3yOs6kDVmEN0tQAuf/UBION_final_project/data


# show_result, store_result

In [ ]:
def show_result(y_test, prediction, prob, best):
  global condition
  CM = confusion_matrix(y_test, prediction) #Confusion_matrix
  A = round( accuracy_score(y_test,prediction),3)  # 정확도
  P = round( precision_score(y_test, prediction),3)  #정밀도
  R = round( recall_score(y_test, prediction), 3) #재현율
  F = round( f1_score(y_test, prediction), 3) #f1-score
  S = round( roc_auc_score(y_test, prob, multi_class='ovr'), 3) #ROC_AUC Score

  condition.append(CM)
  condition.append(A)
  condition.append(P)
  condition.append(R)
  condition.append(F)
  condition.append(S)
  condition.append(best)

In [ ]:
# grid_result_auc = pd.DataFrame(columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score','ROC_AUC', 'hyperparameter 변경', 'cut-off'])
# grid_result_auc.to_csv('grid_result_auc.csv', index=False)

In [ ]:
def store_result(condition):
    grid_result_auc = pd.read_csv('grid_result_auc.csv')

    condition= pd.DataFrame({len(grid_result_auc): condition}).transpose()
    condition.columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score', 'ROC_AUC','hyperparameter 변경', 'cut-off',]
    

    grid_result_auc = pd.concat([grid_result_auc, condition], axis=0, ignore_index=True)

    grid_result_auc.to_csv('grid_result_auc.csv', index=False)

# Excute_model

In [ ]:
def change_cutoff(result_df):
    value_00 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_01 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 1)]['부도_prob'].min()
    value_10 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_11 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 1)]['부도_prob'].min()

    if value_10 < value_00:
        cutoff = value_11
        result_df.loc[(result_df['Actual']==0) & (result_df['prediction']== 1) & (result_df['부도_prob']<cutoff) , 'prediction'] = 0
    
    if value_11 > value_01:
        cutoff= value_00
        result_df.loc[(result_df['Actual']==1) & (result_df['prediction']== 0) & (result_df['부도_prob']>cutoff) , 'prediction'] = 1

    return result_df

In [ ]:
def Excute_lg_grid (X_train, y_train, X_test, y_test):
    # 하이퍼 파라미터 설정 
    parameters = {'C':[0.001, 0.01, 0.1, 1, 10, 100]}
    # 모델 설정
    estimator = LogisticRegression(random_state=0, max_iter=10000)

    #모델 구동
    grid_lr = GridSearchCV(estimator, param_grid=parameters, cv = 5, scoring = 'roc_auc', refit=True)
    grid_lr.fit(X_train, y_train)

    # 최적의 파라미터 조합 도출
    best = grid_lr.best_params_

    # 확률 및 예측값 도출
    prediction = grid_lr.predict(X_test)
    prediction_prob = grid_lr.predict_proba(X_test) 

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

In [ ]:
# def Excute_sgd_change(X_train, y_train, X_test, y_test):
#     sgd = SGDClassifier(random_state=0)
#     sgd.fit(X_train, y_train)
#     prediction = sgd.predict(X_test)
#     prediction_prob =sgd.predict_proba(X_test)

#     y_test = pd.Series(y_test.values)
#     y_test = y_test.to_frame()

#     result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
#     result_df['prediction'] = prediction
#     result_df['Actual'] = y_test


#     show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_svm_grid(X_train, y_train, X_test, y_test):
    parameters = { 'C':[0.1,10,100,1000],'kernel':['rbf','poly','sigmoid','linear'],'degree':[1,2,3,4,5,6],'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
    estimator = SVC(random_state=0, probability = True)

    grid_svm = GridSearchCV(estimator, param_grid=parameters, cv = 5, scoring = 'roc_auc', refit=True)
    grid_svm.fit(X_train, y_train)

    best = grid_svm.best_params_

    prediction = grid_svm.predict(X_test)
    prediction_prob =grid_svm.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

In [ ]:
def Excute_pr_grid(X_train, y_train, X_test, y_test):
    dt2 = DecisionTreeClassifier(max_depth=2)
    dt2.fit(X_train, y_train)
    prediction = dt2.predict(X_test)
    prediction_prob =dt2.predict_proba(X_test)

    best = None

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

In [ ]:
def Excute_dt_grid(X_train, y_train, X_test, y_test): 
    parameters={"min_samples_leaf":[1, 2, 3, 4, 5, 6, 7, 8, 9], "max_depth":[2, 3, 4, 5, 6, None], "min_samples_split":[2, 3, 4, 5, 6, 7, 8, 9, 10] }
    estimator = DecisionTreeClassifier(random_state=0)

    grid_dt = GridSearchCV(estimator, param_grid=parameters, cv = 5, scoring = 'roc_auc', refit=True)
    grid_dt.fit(X_train, y_train)

    best = grid_dt.best_params_

    prediction = grid_dt.predict(X_test)
    prediction_prob =grid_dt.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

In [ ]:
def Excute_knn_grid(X_train, y_train, X_test, y_test):
    parameters = {'n_neighbors' : list(range(1,20)), 'weights' : ["uniform", "distance"], 'metric' : ['euclidean', 'manhattan', 'minkowski']}
    estimator = KNeighborsClassifier()

    grid_knn = GridSearchCV(estimator, param_grid=parameters, cv = 5, scoring = 'roc_auc', refit=True)
    grid_knn.fit(X_train, y_train)

    best = grid_knn.best_params_

    prediction = grid_knn.predict(X_test)
    prediction_prob =grid_knn.predict_proba(X_test)
    

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

In [ ]:
def Excute_rf_grid(X_train, y_train, X_test, y_test):
    parameters={"min_samples_leaf":[1, 2, 3, 4, 5, 6, 7, 8, 9], "max_depth":[2, 3, 4, 5, 6, 8, 10, 12, None], "min_samples_split":[2, 3, 4, 5, 6, 7, 8, 9, 10]}
    estimator= RandomForestClassifier(random_state=0)

    grid_rfc = GridSearchCV(estimator, param_grid=parameters, cv = 5, scoring = 'roc_auc', refit=True)
    grid_rfc.fit(X_train, y_train)

    best = grid_rfc.best_params_

    prediction = grid_rfc.predict(X_test)
    prediction_prob =grid_rfc.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    show_result(result_df['Actual'], result_df['prediction'], result_df['부도_prob'], best)

# sampling

In [ ]:
def RUSE(X_train, y_train):
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler()
  
  X_undersampled, y_undersampled = rus.fit_resample(X_train, y_train)
  
  return X_undersampled, y_undersampled

In [ ]:
def ROSE(X_train, y_train):
  from imblearn.over_sampling import RandomOverSampler
  ros = RandomOverSampler()

  X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train)
  
  return X_oversampled, y_oversampled

In [ ]:
def SMOTE(X_train, y_train):
  from imblearn.over_sampling import SMOTE
  smote = SMOTE()
  X_smoted, y_smoted = smote.fit_resample(X_train,y_train)
  
  return X_smoted, y_smoted

# train/test/split

In [ ]:
def split_by_Y(df):
    after_2018  = df.loc[df['회계년도'] >= 2018]['차기부도여부'].value_counts()[1]
    before_2018 = df.loc[df['회계년도'] <  2018]['차기부도여부'].value_counts()[1]
    SIZE = after_2018/len(df)

    test = df[df['회계년도'] >= 2018]
    train = df[df['회계년도'] < 2018]
    
    test.drop(columns = ['회계년도'], inplace=True)
    train.drop(columns= ['회계년도'], inplace=True)

    X_train = train.drop(['차기부도여부'], axis= 1)
    y_train = train['차기부도여부']

    X_test = test.drop(['차기부도여부'], axis=1)
    y_test = test['차기부도여부']

    return X_train, y_train, X_test, y_test

In [ ]:
def split_by_R(df):
    from sklearn.model_selection import train_test_split
    
    data = df.drop(['차기부도여부'], axis= 1)
    target = df['차기부도여부']

    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.178, shuffle=True, stratify=target, random_state=34)
    X_test.drop(columns = ['회계년도'], inplace=True)
    X_train.drop(columns= ['회계년도'], inplace=True)

    return X_train, y_train, X_test, y_test

# main

In [ ]:
def RUN_ALL(filename, seleted_variables, train_test_split_method , sampling_method, model_name):
    df = pd.read_csv(filename+'.csv')
    global condition

    df = df[seleted_variables]

    if train_test_split_method == 'Y' :
        X_train, y_train, X_test, y_test = split_by_Y(df)
        print(X_train.head(2))
    elif train_test_split_method == 'R' :
        X_train, y_train, X_test, y_test = split_by_R(df)

    if sampling_method == 'RUSE' :
        sampled_X_train, sampled_y_train = RUSE( X_train, y_train )
    elif sampling_method =='ROSE' :
        sampled_X_train, sampled_y_train = ROSE( X_train, y_train )
    elif sampling_method =='SMOTE' :
        sampled_X_train, sampled_y_train = SMOTE( X_train, y_train)
    elif sampling_method =='None':
        sampled_X_train, sampled_y_train = X_train, y_train


    if model_name =='lg':
        Excute_lg_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    # elif model_name =='sgd':
    #     Excute_sgd_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='knn':
        Excute_knn_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='svm':
        Excute_svm_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='dt':
        Excute_dt_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='pr':
        Excute_pr_grid(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='rf':
        Excute_rf_grid(sampled_X_train, sampled_y_train, X_test, y_test)

In [ ]:
target = pd.read_csv('tarrget_92_16.csv')
target= target[target['적용모델']!='sgd']

In [ ]:
for idx, col in target.iterrows():
    condition = col[:8].to_list()

    seleted_variables = condition[7]
    seleted_variables = re.sub("\[|\]|\'|\'|", "",seleted_variables)
    seleted_variables = list(seleted_variables.split(', '))
    seleted_variables.append('차기부도여부')
    seleted_variables.append('회계년도')

    print("***", condition[:7])
    RUN_ALL(condition[0], seleted_variables, condition[1] , condition[5], condition[6])
    condition.append('N')
    store_result(condition)

*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'RUSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'RUSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'ROSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'ROSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'RUSE', 'lg']
*** ['diff_df', 'R', 't_Y', 'None', 'v_N', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'rf']
*** ['basic_final_df', 'R', 't_Y', 'None', 'v_Y', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_Y', 'RUSE', 'rf']
*** ['diff_df', 'R', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_N', 'RO